In [2]:
!pip install simpletransformers

In [1]:
import pandas as pd
import math
import numpy as np
from seqeval.metrics import f1_score

from seqeval.metrics import classification_report, accuracy_score, f1_score
from tqdm import tqdm, trange

In [3]:
dataset_train_path = './data/droner_train_4.csv'
dataset_test_path = './data/droner_test_4.csv'
dataset_train = pd.read_csv(dataset_train_path, encoding= 'unicode_escape')
dataset_test = pd.read_csv(dataset_test_path, encoding= 'unicode_escape')
dataset_train

,Unnamed: 0,drone_model,dataset,controller,sentence_id,words,labels
0,0,DJI_Inspire_1,df010,mobile_iOS,1,Taking,B-ACTION
1,1,DJI_Inspire_1,df010,mobile_iOS,1,Off,I-ACTION
2,2,DJI_Inspire_1,df010,mobile_iOS,1,.,O
3,3,DJI_Inspire_1,df010,mobile_iOS,2,Taking,B-ACTION
4,4,DJI_Inspire_1,df010,mobile_iOS,2,Off,I-ACTION
...,...,...,...,...,...,...,...
9257,12081,DJI_S1000+,df043,mobile_iOS_backup,1816,Please,O
9258,12082,DJI_S1000+,df043,mobile_iOS_backup,1816,fly,O
9259,12083,DJI_S1000+,df043,mobile_iOS_backup,1816,with,O
9260,12084,DJI_S1000+,df043,mobile_iOS_backup,1816,caution,O


In [4]:
dataset_train = dataset_train[['sentence_id', 'words', 'labels']]
dataset_test = dataset_test[['sentence_id', 'words', 'labels']]
dataset_train

,sentence_id,words,labels
0,1,Taking,B-ACTION
1,1,Off,I-ACTION
2,1,.,O
3,2,Taking,B-ACTION
4,2,Off,I-ACTION
...,...,...,...
9257,1816,Please,O
9258,1816,fly,O
9259,1816,with,O
9260,1816,caution,O


In [40]:
dataset_test

,sentence_id,words,labels
0,1627,Backward,B-FUNCTION
1,1627,Obstacle,I-FUNCTION
2,1627,Sensing,I-FUNCTION
3,1627,is,O
4,1627,not,O
...,...,...,...
4090,1883,Please,O
4091,1883,fly,O
4092,1883,with,O
4093,1883,caution,O


In [5]:
dataset_train["labels"].value_counts()

O              5099
I-PARAMETER     985
B-PARAMETER     897
B-ISSUE         697
B-COMPONENT     316
I-FUNCTION      272
I-ISSUE         244
B-FUNCTION      218
B-ACTION        185
I-COMPONENT     144
I-ACTION         98
I-STATE          54
B-STATE          53
Name: labels, dtype: int64

In [6]:
dataset_test["labels"].value_counts()

O              2949
I-PARAMETER     195
B-PARAMETER     153
I-ISSUE         143
B-COMPONENT     132
B-ISSUE         101
B-ACTION        100
I-FUNCTION       86
B-FUNCTION       77
B-STATE          71
I-STATE          48
I-ACTION         25
I-COMPONENT      15
Name: labels, dtype: int64

In [7]:
label = dataset_train["labels"].unique().tolist()
label

['B-ACTION',
 'I-ACTION',
 'O',
 'B-PARAMETER',
 'I-PARAMETER',
 'B-STATE',
 'I-STATE',
 'B-COMPONENT',
 'B-ISSUE',
 'I-ISSUE',
 'B-FUNCTION',
 'I-FUNCTION',
 'I-COMPONENT']

In [2]:
from simpletransformers.ner import NERModel, NERArgs

args = NERArgs()
args.num_train_epochs = 3
args.learning_rate = 0.0001 # 0.0001  5e-5, 3e-5, 2e-5
args.overwrite_output_dir = True
args.train_batch_size = 16
args.eval_batch_size = 8
args.do_lower_case = False
# args.wandb_project = "dfler" # optional

In [9]:
model = NERModel("bert", "bert-base-cased", labels=label, args =args, use_cuda=True)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [11]:
model.train_model(dataset_train, eval_data = dataset_test, acc=accuracy_score)

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ········


wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\s3/.netrc


Running Epoch 0 of 3:   0%|          | 0/89 [00:00<?, ?it/s]

C:\ProgramData\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Running Epoch 1 of 3:   0%|          | 0/89 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/89 [00:00<?, ?it/s]

(267, 0.1595088865222978)

In [12]:
result, model_outputs, preds_list = model.eval_model(dataset_test)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/55 [00:00<?, ?it/s]

Training loss,█▁▁▁▁
global_step,▁▃▅▆█
lr,█▆▅▃▁
Training loss,0.00168
global_step,250
lr,1e-05


wandb: WARNING wandb.plots.* functions are deprecated and will be removed in a future release. Please use wandb.plot.* instead.


In [13]:
result

{'eval_loss': 0.15393207139145074,
 'precision': 0.8913362701908958,
 'recall': 0.8900293255131965,
 'f1_score': 0.8906823184152604}

In [41]:
preds_list

[['B-PARAMETER', 'I-PARAMETER', 'O', 'O', 'B-STATE'],
 ['O', 'B-PARAMETER', 'I-PARAMETER', 'O', 'O', 'O', 'O', 'O'],
 ['B-PARAMETER', 'I-PARAMETER', 'O', 'O', 'B-STATE'],
 ['O', 'B-PARAMETER', 'I-PARAMETER', 'O', 'O', 'O', 'O', 'O'],
 ['B-PARAMETER', 'I-PARAMETER', 'O', 'O'],
 ['B-PARAMETER', 'I-PARAMETER', 'O', 'O', 'B-STATE'],
 ['O', 'B-PARAMETER', 'I-PARAMETER', 'O', 'O', 'O', 'O', 'O'],
 ['B-PARAMETER', 'I-PARAMETER', 'O', 'O', 'B-STATE'],
 ['O', 'B-PARAMETER', 'I-PARAMETER', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'B-STATE', 'O', 'I-STATE'],
 ['B-PARAMETER', 'I-PARAMETER', 'O', 'O', 'B-STATE'],
 ['O', 'B-PARAMETER', 'I-PARAMETER', 'O', 'O', 'O', 'O', 'O'],
 ['B-PARAMETER', 'I-PARAMETER', 'O', 'O', 'B-STATE'],
 ['O', 'B-PARAMETER', 'I-PARAMETER', 'O', 'O', 'O', 'O', 'O'],
 ['B-PARAMETER', 'I-PARAMETER', 'O', 'O', 'B-STATE'],
 ['O', 'B-PARAMETER', 'I-PARAMETER', 'O', 'O', 'O', 'O', 'O'],
 ['B-PARAMETER', 'I-PARAMETER', 'O', 'O', 'B-STATE'],
 ['O', 'B-PARAMETER', 'I-PARAMETER', 'O

In [42]:
# Groupby and collect columns
dataset_test_group = dataset_test.groupby(['sentence_id'], as_index=False)['words', 'labels'].agg(lambda x: list(x))
y_test = dataset_test_group['labels']
x_test = dataset_test_group['words']
y_test

0              [B-PARAMETER, I-PARAMETER, O, O, B-STATE]
1      [O, B-PARAMETER, I-PARAMETER, O, B-STATE, O, O...
2              [B-PARAMETER, I-PARAMETER, O, O, B-STATE]
3      [O, B-PARAMETER, I-PARAMETER, O, B-STATE, O, O...
4                       [B-PARAMETER, I-PARAMETER, O, O]
                             ...                        
433    [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...
434    [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...
435    [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...
436    [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...
437    [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...
Name: labels, Length: 438, dtype: object

In [51]:
x_test_list = []
y_pred_list = []
y_test_list = []
sentence_id_list = []
for row in range(0, len(preds_list)):
    sentence_id_list = np.concatenate((sentence_id_list, list(np.repeat(row+1, len(x_test[row])))), axis=0)
    y_pred_list = np.concatenate((y_pred_list, preds_list[row]), axis=0)
    y_test_list = np.concatenate((y_test_list, y_test[row]), axis=0)
    x_test_list = np.concatenate((x_test_list, x_test[row]), axis=0)
y_pred_list

array(['B-PARAMETER', 'I-PARAMETER', 'O', ..., 'O', 'O', 'O'],
      dtype='<U32')

In [52]:
data_prediction = pd.DataFrame({'sentence_id': sentence_id_list, 'words': x_test_list, 'actual_class': y_test_list, 'predicted_class': y_pred_list})
data_prediction

,sentence_id,words,actual_class,predicted_class
0,1.0,Flight,B-PARAMETER,B-PARAMETER
1,1.0,Mode,I-PARAMETER,I-PARAMETER
2,1.0,Set,O,O
3,1.0,to,O,O
4,1.0,Attitude,B-STATE,B-STATE
...,...,...,...,...
4090,438.0,Please,O,O
4091,438.0,fly,O,O
4092,438.0,with,O,O
4093,438.0,caution,O,O


In [53]:
data_prediction.to_excel('./train_results/prediction_result.xlsx')

In [17]:
confusion_matrix = pd.crosstab(data_prediction['predicted_class'], data_prediction['actual_class'])
confusion_matrix

actual_class,B-ACTION,B-COMPONENT,B-FUNCTION,B-ISSUE,B-PARAMETER,B-STATE,I-ACTION,I-COMPONENT,I-FUNCTION,I-ISSUE,I-PARAMETER,I-STATE,O
predicted_class,,,,,,,,,,,,,
B-ACTION,60,1,3,0,0,1,0,0,0,0,0,0,6
B-COMPONENT,0,131,0,0,0,0,0,0,0,0,0,0,1
B-FUNCTION,0,0,72,0,0,0,0,0,0,0,0,0,2
B-ISSUE,18,0,0,99,0,0,0,0,0,2,0,0,0
B-PARAMETER,0,0,0,2,152,2,0,0,0,0,0,0,2
B-STATE,0,0,0,0,0,56,0,0,0,0,0,0,0
I-ACTION,0,0,0,0,0,0,25,0,0,0,0,0,0
I-COMPONENT,0,0,0,0,0,0,0,15,3,0,0,0,0
I-FUNCTION,1,0,2,0,0,0,0,0,74,0,0,2,3


In [18]:
confusion_matrix.to_excel('./train_results/confusion_matris_gpu.xlsx')

In [19]:
y_test = data_prediction['actual_class'].to_list()
y_pred = data_prediction['predicted_class'].to_list()

In [20]:
from sklearn.metrics import classification_report
pd.DataFrame(classification_report(y_test, y_pred, output_dict=True, labels=['O', 'B-ACTION', 'I-ACTION', 'B-COMPONENT', 'I-COMPONENT', 'B-FUNCTION', 'I-FUNCTION', 'B-ISSUE', 'I-ISSUE', 'B-STATE', 'I-STATE', 'B-PARAMETER', 'I-PARAMETER'])).T

,precision,recall,f1-score,support
O,0.985863,0.993218,0.989527,2949.000000
B-ACTION,0.845070,0.600000,0.701754,100.000000
I-ACTION,1.000000,1.000000,1.000000,25.000000
B-COMPONENT,0.992424,0.992424,0.992424,132.000000
I-COMPONENT,0.833333,1.000000,0.909091,15.000000
B-FUNCTION,0.972973,0.935065,0.953642,77.000000
I-FUNCTION,0.902439,0.860465,0.880952,86.000000
B-ISSUE,0.831933,0.980198,0.900000,101.000000
I-ISSUE,0.972414,0.986014,0.979167,143.000000
B-STATE,1.000000,0.788732,0.881890,71.000000


In [22]:
from seqeval.metrics import classification_report
pd.DataFrame(classification_report([y_test], [y_pred], mode=None, scheme='IOB2', output_dict=True)).T

,precision,recall,f1-score,support
ACTION,0.789474,0.600000,0.681818,100.0
COMPONENT,0.970370,0.992424,0.981273,132.0
FUNCTION,0.873418,0.896104,0.884615,77.0
ISSUE,0.776000,0.960396,0.858407,101.0
PARAMETER,0.944724,0.994709,0.969072,189.0
STATE,0.925373,0.746988,0.826667,83.0
micro avg,0.891336,0.890029,0.890682,682.0
macro avg,0.879893,0.865104,0.866975,682.0
weighted avg,0.891531,0.890029,0.886059,682.0


In [84]:
prediction, model_output = model.predict(["Aircraft is returning to the Home Point Minimum RTH Altitude is 30m You can reset the RTH Altitude in Remote Controller Settings after cancelling RTH"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [88]:
prediction[0]

[{'Aircraft': 'B-COMPONENT'},
 {'is': 'O'},
 {'returning': 'O'},
 {'to': 'O'},
 {'the': 'O'},
 {'Home': 'B-PARAMETER'},
 {'Point': 'I-PARAMETER'},
 {'Minimum': 'B-PARAMETER'},
 {'RTH': 'I-PARAMETER'},
 {'Altitude': 'I-PARAMETER'},
 {'is': 'O'},
 {'30m': 'O'},
 {'You': 'O'},
 {'can': 'O'},
 {'reset': 'O'},
 {'the': 'O'},
 {'RTH': 'B-PARAMETER'},
 {'Altitude': 'I-PARAMETER'},
 {'in': 'O'},
 {'Remote': 'B-FUNCTION'},
 {'Controller': 'I-FUNCTION'},
 {'Settings': 'I-FUNCTION'},
 {'after': 'O'},
 {'cancelling': 'O'},
 {'RTH': 'B-ACTION'}]

In [83]:
[prediction2], model_output2 = model.predict(["Weak signal Avoid blocking the antennas and keep the antennas parallel to and facing toward the aircraft during flight"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [79]:
prediction2

[{'Weak': 'B-ISSUE'},
 {'signal': 'I-ISSUE'},
 {'Avoid': 'O'},
 {'blocking': 'O'},
 {'the': 'O'},
 {'antennas': 'B-COMPONENT'},
 {'and': 'O'},
 {'keep': 'O'},
 {'the': 'O'},
 {'antennas': 'B-COMPONENT'},
 {'parallel': 'O'},
 {'to': 'O'},
 {'and': 'O'},
 {'facing': 'O'},
 {'toward': 'O'},
 {'the': 'O'},
 {'aircraft': 'B-COMPONENT'},
 {'during': 'O'},
 {'flight': 'O'}]

In [3]:
# Load the fine-tuned model
droner = NERModel(
    "bert", "outputs", use_cuda=True
)

In [4]:
# Predict using the loaded model
[prediction3], model_output3 = droner.predict(["Compass abnormal. Solution: 1. Ensure there are no magnets or metal objects near the aircraft. The ground or walls may contain metal. Move away from sources of interference before attempting flight. 2. Calibrate Compass Before Takeoff."])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
prediction3

[{'Compass': 'B-ISSUE'},
 {'abnormal.': 'I-ISSUE'},
 {'Solution:': 'O'},
 {'1.': 'O'},
 {'Ensure': 'O'},
 {'there': 'O'},
 {'are': 'O'},
 {'no': 'O'},
 {'magnets': 'B-COMPONENT'},
 {'or': 'O'},
 {'metal': 'B-COMPONENT'},
 {'objects': 'B-COMPONENT'},
 {'near': 'O'},
 {'the': 'O'},
 {'aircraft.': 'B-COMPONENT'},
 {'The': 'O'},
 {'ground': 'O'},
 {'or': 'O'},
 {'walls': 'O'},
 {'may': 'O'},
 {'contain': 'O'},
 {'metal.': 'O'},
 {'Move': 'O'},
 {'away': 'O'},
 {'from': 'O'},
 {'sources': 'O'},
 {'of': 'O'},
 {'interference': 'I-ISSUE'},
 {'before': 'O'},
 {'attempting': 'O'},
 {'flight.': 'O'},
 {'2.': 'O'},
 {'Calibrate': 'B-FUNCTION'},
 {'Compass': 'I-ISSUE'},
 {'Before': 'O'},
 {'Takeoff.': 'O'}]